# 振子2实验：

## 使用SINDY测试

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import pysindy as ps
from pysindy.feature_library import GeneralizedLibrary, PolynomialLibrary, FourierLibrary
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso

# 自定义特征函数
def x_cos_x(x):
    return x * np.cos(x)

def x_sin_x(x):
    return x * np.sin(x)

def exp_x(x):
    return np.exp(x)

# 修复 function_names 参数，确保其为字符串列表
custom_library = ps.CustomLibrary(
    library_functions=[x_cos_x, x_sin_x, exp_x],
    function_names=[lambda x: "x*cos(x)", lambda x: "x*sin(x)", lambda x: "exp(x)"]  # 使用 lambda 确保正确的字符串表示
)

# 1. 读取数据
df = pd.read_csv('./test_ood.csv')
t = df['t'].values
x = df['x'].values
v = df['v'].values
a = df['a'].values

X = np.vstack((t, x, v)).T
y = a.T

# 2. 构建并拟合 SINDy 模型
#    - 用多项式库（degree=5）来捕捉 sin(theta) 展开
#    - 用 STLSQ 稀疏回归
poly_lib = PolynomialLibrary(degree=4)            # [1, x, x^2, x^3]
fourier_lib = FourierLibrary(n_frequencies=2)      #  [1, sin(x), cos(x), sin(2x), cos(2x)]
library = GeneralizedLibrary(libraries=[poly_lib, fourier_lib, custom_library])
optimizer = Lasso(alpha=0.001)  # 设置正则化强度
#optimizer = ps.STLSQ(threshold=0.1, alpha=0.1)  # 稀疏回归
#optimizer = ps.SR3(threshold=0.1)  # 稀疏回归
model = ps.SINDy(feature_library=library, optimizer=optimizer)
model.fit(X, x_dot=a)
model.print()  # 打印识别出的方程

# 使用模型预测
predicted_values = model.predict(X)

# 计算均方误差 (MSE)
mse = mean_squared_error(a, predicted_values)
print(f"均方误差 (MSE): {mse}")



c:\anacoda\envs\pytorch\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.132e+01, tolerance: 9.700e-01
  model = cd_fast.enet_coordinate_descent(


(x0)' = 0.004 x0 + 0.001 x0 x1 + -0.985 x1 x2 + -0.003 x0 x1 x2 + -0.500 x2^3 + -0.003 x0 x1 x2^2 + -0.003 x2^4 + 0.303 sin(1 x0) + 0.002 cos(1 x0) + -0.087 sin(2 x1) + -0.010 cos(2 x2) + -4.816 exp(x) + 1.908
均方误差 (MSE): 8.406787789445808


# 验证集损失

In [12]:
# 1. 读取数据
df = pd.read_csv('./train.csv')
t = df['t'].values
x = df['x'].values
v = df['v'].values
a = df['a'].values

X = np.vstack((t, x, v)).T
y = a.T

predicted_values = model.predict(X)

# 计算均方误差 (MSE)
mse = mean_squared_error(a, predicted_values)
print(f"均方误差 (MSE): {mse}")

均方误差 (MSE): 3.218739465339395e+20


# 使用PYSR测试

In [13]:
import numpy as np
import pandas as pd
from pysr import PySRRegressor
import matplotlib.pyplot as plt

# 1. 读取数据
df = pd.read_csv('./test_ood.csv')
t = df['t'].values
x = df['x'].values
v = df['v'].values
a = df['a'].values

X = np.vstack((t, x, v)).T
y = a.T

# 定义 PySR 模型
model = PySRRegressor(
    niterations=100,  # 迭代次数
    populations=30,  # 种群数量
    binary_operators=["+", "-", "*", "/","^"],  # 二元操作符
    unary_operators=["sin", "cos","exp"],  # 一元操作符
    nested_constraints={
                "sin": {"cos": 0,"sin":0},  # 禁止 sin(cos(x))
                "exp": {"exp": 0,"cos": 0,"sin":0},  # 禁止 exp(exp(x))
                "cos": {"sin": 0,"cos": 0},
            },
    constraints={
        "^": (-1, 2)  # 再次尝试限制指数为常数
    },
    elementwise_loss="L2DistLoss()",  # 损失函数
    parallelism='multithreading',  # 启用多线程
    verbosity=1,  # 显示详细日志 (verbosity 而不是 verbose)
)

# 拟合模型
print("Fitting model...")
model.fit(X, y)

# 输出最佳表达式
for i, expr in enumerate(model.equations_):
    print(f"Best Expression for output {i+1}: {expr}")



Fitting model...


c:\anacoda\envs\pytorch\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 2.230e+04
Progress: 124 / 3000 total iterations (4.133%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           9.700e-01  1.594e+01  y = -0.028345
3           7.507e-02  1.279e+00  y = x₁ * -4.7665
5           6.338e-02  8.457e-02  y = x₁ * (-4.7117 - x₁)
7           5.952e-02  3.142e-02  y = (x₁ * 1.9705) * (-2.4032 - x₁)
8           5.910e-02  7.233e-03  y = x₁ * (-1.9953 * (exp(x₁) + 1.325))
17          5.015e-02  1.824e-02  y = ((((x₁ + -0.54187) * sin(x₀)) * sin(-0.2271)) - x₁) + ...
                                      (-1.7729 * (x₁ + x₁))
23          4.953e-02  2.069e-03  y = (((((((cos(x₂ / -0.35998) * -0.25924) / exp(x₀)) - x₁)...
                                       - -1.6019e-05) / 0.30553) / 0.68986) - x₁) + (x₁ + -0.077...
             

[ Info: Final population:
[ Info: Results saved to:


───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           9.700e-01  1.594e+01  y = -0.028345
3           7.507e-02  1.279e+00  y = x₁ * -4.7665
5           6.332e-02  8.509e-02  y = x₁ * (-4.7522 - x₁)
6           5.953e-02  6.174e-02  y = (1.004 - exp(x₁)) * 4.6356
7           5.875e-02  1.321e-02  y = (-4.76 - (x₁ + x₂)) * x₁
8           3.817e-02  4.312e-01  y = (x₁ * -4.9815) + (sin(x₀) * 0.2809)
10          2.447e-02  2.223e-01  y = (sin(x₀) * 0.28823) + ((-4.9727 - x₁) * x₁)
12          1.846e-02  1.409e-01  y = (x₁ * (-4.9636 - (x₁ / 0.44818))) + (sin(x₀) * 0.29743...
                                      )
13          1.794e-02  2.887e-02  y = ((x₂ - sin(x₀)) * -0.26624) + (x₁ * (-3.9374 - exp(x₁)...
                                      ))
14          1.034e-02  5.506e-01  y = (((x₂ - sin(x₀)) * 0.057717) + x₁) * (-5.0094 - (x₁ / ...
                                      0.38976))

In [14]:
# 1. 读取数据
df = pd.read_csv('./train.csv')
t = df['t'].values
x = df['x'].values
v = df['v'].values
a = df['a'].values

X = np.vstack((t, x, v)).T
y = a.T

predicted_values = model.predict(X)

# 计算均方误差 (MSE)
mse = mean_squared_error(a, predicted_values)
print(f"均方误差 (MSE): {mse}")

c:\anacoda\envs\pytorch\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


均方误差 (MSE): 2.242395226196439e-05
